In [49]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.14.0") // Use the latest version
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.15.2")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.7.3")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")


In [50]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
}


In [51]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.lookup.PlanningId

data class Agent(
    @PlanningId
    val name: String,
    val homeLocation: Location,
    val capacity: Int,
    val skills: List<String>,
    var currentLocation: Location = homeLocation, // To track agent's current location
    var availableFrom: Int = 0 // Time when the agent is available
) {
    override fun toString(): String = name
}


In [52]:
data class Meeting(
    @PlanningId
    val name: String,
    val location: Location,
    val requiredSkill: String,
    val timeWindow: List<Int>, // [start, end]
    val duration: Int
) {
    override fun toString(): String = name
}


In [53]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Assignment(
    val meeting: Meeting
) {
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"])
    var agent: Agent? = null

    var startTime: Int? = null // Time when the agent starts the meeting

    override fun toString(): String = "$meeting assigned to ${agent?.name ?: "Unassigned"}"
}


In [54]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Assignment(
    val meeting: Meeting
) {
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"])
    var agent: Agent? = null

    var startTime: Int? = null // Time when the agent starts the meeting

    override fun toString(): String = "$meeting assigned to ${agent?.name ?: "Unassigned"}"
}


In [55]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore

@PlanningSolution
data class Schedule(
    val name: String,

    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    val meetings: List<Meeting>,

    @PlanningEntityCollectionProperty
    val assignments: List<Assignment>,

    @PlanningScore
    var score: HardSoftLongScore? = null
) {
    // No-arg constructor required for Timefold
    constructor() : this("", emptyList(), emptyList(), emptyList(), null)
}
